In [2]:
import pandas as pd
import numpy as np

from rl_lib.swiss_round.environment import SwissRoundEnv
from rl_lib.swiss_round.agent import DQNAgent

%reload_ext autoreload
%autoreload 2

# Utils 

In [3]:
def probability_tables(team_strengths, max_draw_probability):
    index = range(len(team_strengths))
    wps = []
    dps = []
    lps = []
    for ts1 in team_strengths :
        twps = []
        tdps = []
        tlps = []
        for ts2 in team_strengths :
            strength_diff = ts1 - ts2
            tmp_win_prob = 1 / (1 + np.exp(-strength_diff))
            tmp_loss_prob = 1 / (1 + np.exp(+strength_diff))
            tmp_draw_prob = max_draw_probability * np.exp(-abs(strength_diff))
            # Softmax
            win_prob = tmp_win_prob / (tmp_win_prob + tmp_draw_prob + tmp_loss_prob)
            draw_prob = tmp_draw_prob / (tmp_win_prob + tmp_draw_prob + tmp_loss_prob)
            loss_prob = tmp_loss_prob / (tmp_win_prob + tmp_draw_prob + tmp_loss_prob)  
            
            twps.append(win_prob)
            tdps.append(draw_prob)
            tlps.append(loss_prob)
        wps.append(twps)
        dps.append(tdps)
        lps.append(tlps)
    return pd.DataFrame(wps, index=index, columns = index), pd.DataFrame(dps, index=index, columns = index),pd.DataFrame(lps, index=index, columns = index)        

In [4]:
def check_probability(team_strengths, max_draw_probability):
    wp, dp, lp = probability_tables(team_strengths=team_strengths, max_draw_probability=max_draw_probability)
    df = wp+dp+lp
    values_array = df.to_numpy()
    target_array = np.full_like(values_array, 1)

    return np.allclose(values_array, target_array, rtol=10e-5, atol=10e-8)

# Environement

In [32]:
n_teams = 18
max_str = 18
threshold_ranks = [4,12]
agent_id = threshold_ranks[-1] 
team_strengths = np.linspace(max_str,0,n_teams)
#team_strengths = [4 * 0.9 ** i for i in range(n_teams)]
mdp = 0.5

print(np.array(team_strengths).round(2))
wp, dp, lp = probability_tables(team_strengths=team_strengths, max_draw_probability=mdp)
pd.concat([wp.loc[[agent_id]].rename(index={agent_id:'Win'}),
           dp.loc[[agent_id]].rename(index={agent_id:'Draw'}),
           lp.loc[[agent_id]].rename(index={agent_id:'Loss'}),
           ]).T.round(2)

[18.   16.94 15.88 14.82 13.76 12.71 11.65 10.59  9.53  8.47  7.41  6.35
  5.29  4.24  3.18  2.12  1.06  0.  ]


,Win,Draw,Loss
0,0.00,0.00,1.00
1,0.00,0.00,1.00
2,0.00,0.00,1.00
3,0.00,0.00,1.00
4,0.00,0.00,1.00
5,0.00,0.00,1.00
6,0.00,0.00,1.00
7,0.00,0.00,0.99
8,0.01,0.01,0.98
9,0.04,0.02,0.94


In [33]:
n_rounds = 6
bonus_points = [30,20]
#Agent_id just below last threshold
n_baselines_simu = 2000
name = f'lin_{max_str}'
print(f"Valid probability set-up : {check_probability(team_strengths, mdp)}")

env = SwissRoundEnv(
    n_teams=n_teams,
    n_rounds=n_rounds,
    name=name,
    team_strengths=team_strengths,
    threshold_ranks=threshold_ranks,
    bonus_points=bonus_points,
    agent_id=agent_id,
    max_draw_probability=mdp
)

Valid probability set-up : True


#### Detailed tables

In [7]:
wp.round(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.33,0.40,0.47,0.54,0.60,0.66,0.72,0.76,0.81,0.84,0.87,0.90,0.92,0.93,0.95,0.96,0.97,0.97
1,0.32,0.33,0.40,0.47,0.54,0.60,0.66,0.72,0.76,0.81,0.84,0.87,0.90,0.92,0.93,0.95,0.96,0.97
2,0.29,0.32,0.33,0.40,0.47,0.54,0.60,0.66,0.72,0.76,0.81,0.84,0.87,0.90,0.92,0.93,0.95,0.96
3,0.27,0.29,0.32,0.33,0.40,0.47,0.54,0.60,0.66,0.72,0.76,0.81,0.84,0.87,0.90,0.92,0.93,0.95
4,0.23,0.27,0.29,0.32,0.33,0.40,0.47,0.54,0.60,0.66,0.72,0.76,0.81,0.84,0.87,0.90,0.92,0.93
5,0.20,0.23,0.27,0.29,0.32,0.33,0.40,0.47,0.54,0.60,0.66,0.72,0.76,0.81,0.84,0.87,0.90,0.92
6,0.17,0.20,0.23,0.27,0.29,0.32,0.33,0.40,0.47,0.54,0.60,0.66,0.72,0.76,0.81,0.84,0.87,0.90
7,0.15,0.17,0.20,0.23,0.27,0.29,0.32,0.33,0.40,0.47,0.54,0.60,0.66,0.72,0.76,0.81,0.84,0.87
8,0.12,0.15,0.17,0.20,0.23,0.27,0.29,0.32,0.33,0.40,0.47,0.54,0.60,0.66,0.72,0.76,0.81,0.84
9,0.10,0.12,0.15,0.17,0.20,0.23,0.27,0.29,0.32,0.33,0.40,0.47,0.54,0.60,0.66,0.72,0.76,0.81


In [8]:
dp.round(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.33,0.28,0.24,0.20,0.16,0.13,0.11,0.09,0.07,0.06,0.05,0.04,0.03,0.02,0.02,0.01,0.01,0.01
1,0.28,0.33,0.28,0.24,0.20,0.16,0.13,0.11,0.09,0.07,0.06,0.05,0.04,0.03,0.02,0.02,0.01,0.01
2,0.24,0.28,0.33,0.28,0.24,0.20,0.16,0.13,0.11,0.09,0.07,0.06,0.05,0.04,0.03,0.02,0.02,0.01
3,0.20,0.24,0.28,0.33,0.28,0.24,0.20,0.16,0.13,0.11,0.09,0.07,0.06,0.05,0.04,0.03,0.02,0.02
4,0.16,0.20,0.24,0.28,0.33,0.28,0.24,0.20,0.16,0.13,0.11,0.09,0.07,0.06,0.05,0.04,0.03,0.02
5,0.13,0.16,0.20,0.24,0.28,0.33,0.28,0.24,0.20,0.16,0.13,0.11,0.09,0.07,0.06,0.05,0.04,0.03
6,0.11,0.13,0.16,0.20,0.24,0.28,0.33,0.28,0.24,0.20,0.16,0.13,0.11,0.09,0.07,0.06,0.05,0.04
7,0.09,0.11,0.13,0.16,0.20,0.24,0.28,0.33,0.28,0.24,0.20,0.16,0.13,0.11,0.09,0.07,0.06,0.05
8,0.07,0.09,0.11,0.13,0.16,0.20,0.24,0.28,0.33,0.28,0.24,0.20,0.16,0.13,0.11,0.09,0.07,0.06
9,0.06,0.07,0.09,0.11,0.13,0.16,0.20,0.24,0.28,0.33,0.28,0.24,0.20,0.16,0.13,0.11,0.09,0.07


In [9]:
lp.round(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.33,0.32,0.29,0.27,0.23,0.20,0.17,0.15,0.12,0.10,0.08,0.07,0.05,0.04,0.04,0.03,0.02,0.02
1,0.40,0.33,0.32,0.29,0.27,0.23,0.20,0.17,0.15,0.12,0.10,0.08,0.07,0.05,0.04,0.04,0.03,0.02
2,0.47,0.40,0.33,0.32,0.29,0.27,0.23,0.20,0.17,0.15,0.12,0.10,0.08,0.07,0.05,0.04,0.04,0.03
3,0.54,0.47,0.40,0.33,0.32,0.29,0.27,0.23,0.20,0.17,0.15,0.12,0.10,0.08,0.07,0.05,0.04,0.04
4,0.60,0.54,0.47,0.40,0.33,0.32,0.29,0.27,0.23,0.20,0.17,0.15,0.12,0.10,0.08,0.07,0.05,0.04
5,0.66,0.60,0.54,0.47,0.40,0.33,0.32,0.29,0.27,0.23,0.20,0.17,0.15,0.12,0.10,0.08,0.07,0.05
6,0.72,0.66,0.60,0.54,0.47,0.40,0.33,0.32,0.29,0.27,0.23,0.20,0.17,0.15,0.12,0.10,0.08,0.07
7,0.76,0.72,0.66,0.60,0.54,0.47,0.40,0.33,0.32,0.29,0.27,0.23,0.20,0.17,0.15,0.12,0.10,0.08
8,0.81,0.76,0.72,0.66,0.60,0.54,0.47,0.40,0.33,0.32,0.29,0.27,0.23,0.20,0.17,0.15,0.12,0.10
9,0.84,0.81,0.76,0.72,0.66,0.60,0.54,0.47,0.40,0.33,0.32,0.29,0.27,0.23,0.20,0.17,0.15,0.12


In [10]:
wp+dp+lp

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


# Simulations

### Logs

In [34]:
display_cols = ['exp_id', 'env_name', 'n_episodes','lr', 'layers', 'dropout', 'avg_test_rewards',
       'std_test_rewards', 'avg_test_gambits', 'std_test_gambits']

In [40]:
cpdf = pd.read_csv('../logs/calmip_logs.csv', index_col=0)
locdf = pd.read_csv('../logs/exp_logs.csv', index_col=0)
ldf = pd.concat([cpdf,locdf]).reset_index(drop=True)
ldf[ldf['env_name']==name].sort_values(by='avg_test_rewards', ascending=False)[display_cols]

,exp_id,env_name,n_episodes,lr,layers,dropout,avg_test_rewards,std_test_rewards,avg_test_gambits,std_test_gambits
20,loc_9,lin_18,20000,0.0005,256_128_64,0.3,16.015,11.254145,0.4235,0.600123


### Baselines Simulations

In [36]:
simulation_wa = env.simulate_n_tournaments(n_baselines_simu,
                                           n_cores = 32, 
                                           policy = 'win_all',
                                           display_results=True)
baseline_reward_wa = simulation_wa.loc[agent_id,'Avg_Reward'] 
baseline_std_wa = simulation_wa.loc[agent_id,'Std_Reward'] 
print(f"Baseline WinAll average reward = {baseline_reward_wa:.1f} ± {baseline_std_wa:.1f}")
simulation_wa.loc[agent_id].round(2)

Simulating tournaments: 100%|██████████| 2000/2000 [00:14<00:00, 140.74it/s]



Simulation Results (from 1961 tournaments):
Team | Strength | Avg Points | Avg Rank | Avg Reward | Std Reward | Top-4 % | Top-12 % | 
----------------------------------------------------------------------------------------
   0 |    18.00 |      16.65 |     1.43 |      66.44 |       3.35 |  99.29% | 100.00% | 
   1 |    16.94 |      14.66 |     2.32 |      62.76 |       8.34 |  93.68% | 100.00% | 
   2 |    15.88 |      13.02 |     3.39 |      57.39 |      12.77 |  81.23% | 100.00% | 
   3 |    14.82 |      11.79 |     4.69 |      47.45 |      16.08 |  52.22% | 100.00% | 
   4 |    13.76 |      11.28 |     5.39 |      42.24 |      15.65 |  36.77% |  99.64% | 
   5 |    12.71 |      10.62 |     6.37 |      36.12 |      12.96 |  18.82% |  99.29% | 
   6 |    11.65 |      10.03 |     7.29 |      32.62 |      10.21 |   9.59% |  98.57% | 
   7 |    10.59 |       9.58 |     8.11 |      30.11 |       8.67 |   4.64% |  95.72% | 
   8 |     9.53 |       9.12 |     9.03 |      28.03 |       8.4

Team          12.00
Strength       5.29
Avg_Points     7.17
Avg_Rank      12.55
Avg_Reward    16.71
Std_Reward    11.41
Top-4 %        0.00
Top-12 %       0.48
Name: 12, dtype: float64

In [37]:
simulation_lf = env.simulate_n_tournaments(n_baselines_simu,n_cores = 32, policy = 'lose_first',display_results=False)
baseline_reward_lf = simulation_lf.loc[agent_id,'Avg_Reward'] 
baseline_std_lf = simulation_lf.loc[agent_id,'Std_Reward'] 
print(f"Baseline Lose First average reward = {baseline_reward_lf:.1f} ± {baseline_std_lf:.1f}")
simulation_lf.loc[agent_id].round(2)

Simulating tournaments: 100%|██████████| 2000/2000 [00:14<00:00, 137.37it/s]

Baseline Lose First average reward = 15.7 ± 11.4


Team          12.00
Strength       5.29
Avg_Points     7.07
Avg_Rank      12.87
Avg_Reward    15.71
Std_Reward    11.36
Top-4 %        0.00
Top-12 %       0.43
Name: 12, dtype: float64

### RL Agent

In [38]:
print(f"Baselines average reward : WinAll = {baseline_reward_wa:.1f} ± {baseline_std_wa:.1f}, "
      f"LoseFirst = {baseline_reward_lf:.1f} ± {baseline_std_lf:.1f}")

Baselines average reward : WinAll = 16.7 ± 11.4, LoseFirst = 15.7 ± 11.4


In [39]:
agent = DQNAgent(env,
                 hidden_dims=[128,64],
                 dropout= 0.4,
                 lr = 0.0005,
                 buffer_size=10000,
                 epsilon_decay=0.9995)
agent.train(n_episodes=20000)
agent.evaluate(n_episodes=2000)

--- Training in progress ---
Episode 100/20000 | Avg Reward: 7.00 ± 7.73 |  Avg nb gambits played 3.71 ± 1.24 | Epsilon: 0.761 | Failed episodes: 2
Episode 200/20000 | Avg Reward: 7.17 ± 8.20 |  Avg nb gambits played 3.56 ± 1.24 | Epsilon: 0.563 | Failed episodes: 3
Episode 300/20000 | Avg Reward: 10.20 ± 9.82 |  Avg nb gambits played 2.99 ± 1.09 | Epsilon: 0.416 | Failed episodes: 4
Episode 400/20000 | Avg Reward: 10.31 ± 9.42 |  Avg nb gambits played 2.39 ± 1.07 | Epsilon: 0.308 | Failed episodes: 4
Episode 500/20000 | Avg Reward: 13.36 ± 10.87 |  Avg nb gambits played 2.30 ± 1.02 | Epsilon: 0.227 | Failed episodes: 6
Episode 600/20000 | Avg Reward: 13.47 ± 10.82 |  Avg nb gambits played 1.79 ± 1.05 | Epsilon: 0.168 | Failed episodes: 7
Episode 700/20000 | Avg Reward: 15.67 ± 11.25 |  Avg nb gambits played 1.58 ± 0.99 | Epsilon: 0.124 | Failed episodes: 7
Episode 800/20000 | Avg Reward: 12.14 ± 10.15 |  Avg nb gambits played 1.37 ± 1.03 | Epsilon: 0.092 | Failed episodes: 10
Episode 

### Verbosed simulation

In [22]:
# Simulate tournament
final_standings = env.simulate_tournament(verbose= True)

print("\nFinal standings (team_id, points, opponent_average):")
for rank, (team_id, points, reward, opp_avg,strength) in enumerate(final_standings, 1):

    print(f"Rank {rank}: Team {team_id} - Strength {strength:.2f} - Points: {points} (reward {reward}) - Opponent Avg: {opp_avg:.2f}")


--- Simulating round n°1 ---
Game : Team 8 (points : 0, strength : 2.12) vs Team 2 (points : 0, strength : 3.53) : Draw
Game : Team 5 (points : 0, strength : 2.82) vs Team 1 (points : 0, strength : 3.76) : Team 1 wins
Game : Team 12 (points : 0, strength : 1.18) vs Team 11 (points : 0, strength : 1.41) : Team 11 wins
Game : Team 17 (points : 0, strength : 0.00) vs Team 9 (points : 0, strength : 1.88) : Team 9 wins
Game : Team 0 (points : 0, strength : 4.00) vs Team 10 (points : 0, strength : 1.65) : Team 10 wins
Game : Team 3 (points : 0, strength : 3.29) vs Team 6 (points : 0, strength : 2.59) : Draw
Game : Team 15 (points : 0, strength : 0.47) vs Team 14 (points : 0, strength : 0.71) : Team 14 wins
Game : Team 13 (points : 0, strength : 0.94) vs Team 7 (points : 0, strength : 2.35) : Team 7 wins
Game : Team 4 (points : 0, strength : 3.06) vs Team 16 (points : 0, strength : 0.24) : Team 4 wins
--- Simulating round n°2 ---
Game : Team 1 (points : 3, strength : 3.76) vs Team 4 (points :